In [2]:
!pip install memory_profiler
!pip install scipy
import numpy as np
import tracemalloc
import networkx as nx
import matplotlib.pyplot as plt
import random
import time

from memory_profiler import profile


from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import johnson
from scipy.optimize import curve_fit

In [5]:
# generate a directed graph with negative weights

def graph(v,e,right):
  num_vertices = v
  num_edges = e
  kolichestvo_negative_weights=0

  g = [[0] * num_vertices for i in range(num_vertices)]
  edges_generated = 0
  while edges_generated < num_edges:
      vertex1 = random.randint(0, num_vertices - 1)
      vertex2 = random.randint(0, num_vertices - 1)

      if vertex1 != vertex2 and g[vertex1][vertex2] == 0:

          g[vertex1][vertex2] = random.randint(-1,right)
          if g[vertex1][vertex2] == -1:
            kolichestvo_negative_weights+=1
          edges_generated += 1
  adj_mtrx=np.array([np.array(xi) for xi in g])
  return g


In [ ]:
g = graph(100,500,10)

In [ ]:
# Define the adjacency matrix as a 2D numpy array
# The (i, j)-th element in the array represents an edge from node i to node j
adjacency_matrix = np.array(g)

# Create a directed graph from the adjacency matrix
G = nx.DiGraph(adjacency_matrix)

# Draw the graph
pos = nx.spring_layout(G)  # You can use different layout algorithms

# You can customize the visualization as needed
nx.draw(G, pos, with_labels=True, node_size=100, node_color='lightblue', font_size=1, font_color='black')
# nx.draw_networkx_edge_labels(G, pos,  font_color='red')
plt.title("Directed Graph with 100 nodes and 500 edges Visualization")
plt.show()

In [ ]:
# Johnson algorithm results
def johnson_alg(g):
  graph = csr_matrix(g)
  dist_matrix, predecessors = johnson(csgraph=graph, directed=True, return_predecessors=True)
  return dist_matrix
johnson_alg(g)

array([[ 0., -1.,  6., ...,  5.,  5., -2.],
       [ 9.,  0.,  8., ...,  7.,  7.,  1.],
       [ 9.,  2.,  0., ..., -1.,  4.,  1.],
       ...,
       [10.,  3., 10., ...,  0.,  7.,  2.],
       [ 8.,  7., 14., ..., 13.,  0.,  6.],
       [ 9.,  1.,  8., ...,  7.,  8.,  0.]])

In [ ]:
# Floyd-Warshall algorithm

def floyd_warshall(graph):
    num_vertices = len(graph)
    dist = [[float('inf') if i != j else 0 for j in range(num_vertices)] for i in range(num_vertices)]

    for u in range(num_vertices):
        for v in range(num_vertices):
            if graph[u][v] != 0:
                dist[u][v] = graph[u][v]

    for k in range(num_vertices):
        for i in range(num_vertices):
            for j in range(num_vertices):
                if dist[i][k] != float('inf') and dist[k][j] != float('inf') and dist[i][k] + dist[k][j] < dist[i][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]

    return dist


# Run the Floyd-Warshall algorithm
shortest_distances = floyd_warshall(adjacency_matrix)

np.array(shortest_distances)

array([[ 0., -1.,  6., ...,  5.,  5., -2.],
       [ 9.,  0.,  8., ...,  7.,  7.,  1.],
       [ 9.,  2.,  0., ..., -1.,  4.,  1.],
       ...,
       [10.,  3., 10., ...,  0.,  7.,  2.],
       [ 8.,  7., 14., ..., 13.,  0.,  6.],
       [ 9.,  1.,  8., ...,  7.,  8.,  0.]])

Time complexity

In [13]:
# Generate a graph without negative weight to prevent negative cycles
def graph2(vertices, edges, max_weight):
    random.seed(2)
    matrix = np.zeros((vertices, vertices), dtype=int)
    count = 0
    while count < edges:
        i, j = random.sample(range(vertices), 2)
        if matrix[i][j] == 0 and i != j:
            weight = random.randint(1, max_weight)
            matrix[i][j] = weight
            matrix[j][i] = weight
            count += 1
    return matrix

In [33]:
def time_taken_for_fw(G_t):
    start = time.time()
    nx.floyd_warshall(G_t)
    end = time.time()
    return end - start

def time_taken_for_johnson(graph):
    start = time.time()
    nx.johnson(G_t)
    end = time.time()
    return end - start

sizes = []
time_fw = []
time_johnson = []

def matrix_to_graph(matrix):
    G = nx.Graph()
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if matrix[i][j]:
                G.add_edge(i, j, weight=matrix[i][j])
    return G

for size in range(20, 110, 2):
    vertices = size
    edges = round(((vertices*(vertices-1))/2)*0.1)
    matrix_t = graph2(vertices, edges, 1000)
    G_t = matrix_to_graph(matrix_t)
    sizes.append(size)
    time_fw.append(time_taken_for_fw(G_t))
    time_johnson.append(time_taken_for_johnson(G_t))

In [ ]:
# Theoretical time complexities:
def fw_theoretical(n, a):
    return a * n**3

def johnson_theoretical(n, b, c):
    return b * n**2 * np.log(n) + c * n **3

In [34]:
params_fw, _ = curve_fit(fw_theoretical, sizes, time_fw)
params_johnson, _ = curve_fit(johnson_theoretical, sizes, time_johnson)

# Plot for Floyd-Warshall
plt.figure(figsize=(10, 5))
plt.scatter(sizes, time_fw, label='Experiment')
plt.plot(sizes, fw_theoretical(np.array(sizes), *params_fw), '-', label='Theoretical',color='green')
plt.xlabel('Number of Vertices')
plt.ylabel('Time (seconds)')
plt.title('Time Complexity of Floyd-Warshall')
plt.legend()
plt.grid(True)
plt.show()

# Plot for Johnson's
plt.figure(figsize=(10, 5))
plt.scatter(sizes, time_johnson, label='Experiment')
plt.plot(sizes, johnson_theoretical(np.array(sizes), *params_johnson), '-', label='Theoretical',color ='green')
plt.xlabel('Number of Vertices')
plt.ylabel('Time (seconds)')
plt.title('Time Complexity of Johnsonâ€™s Algorithm')
plt.legend()
plt.grid(True)
plt.show()

Space complexity

In [ ]:
# importing functions for memory_profiler
from fancy import johnson_space
from fancy import floyd_warshall_space

In [ ]:
# 100 edges

g = graph(100,100,1000)
print('Memory usage of Johnson with 100 edges:')
%load_ext memory_profiler
%mprun -f  johnson_space johnson_space(g)

Memory usage of Johnson with 100 edges:
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
Filename: /Users/polina/Desktop/pythonProject/fancy.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     6     44.1 MiB     44.1 MiB           1   @profile
     7                                         def johnson_space(g):
     8     44.1 MiB      0.0 MiB           1     graph = csr_matrix(g)
     9     44.2 MiB      0.1 MiB           1     dist_matrix, predecessors = johnson(csgraph=graph, directed=True, return_predecessors=True)
    10     44.2 MiB      0.0 MiB           1     return dist_matrix





Filename: /Users/polina/Desktop/pythonProject/venv/lib/python3.11/site-packages/memory_profiler.py

Line #    Mem usage    Increment  Occurrences   Line Contents
  1185     44.1 MiB     44.1 MiB           1               @wraps(wrapped=func)
  1186                                                     def wrapper(*args, **kwargs):
  1187     44.1 MiB      0.0 MiB           1                   prof = get_prof()
  1188     44.2 MiB      0.1 MiB           1                   val = prof(func)(*args, **kwargs)
  1189     44.2 MiB      0.0 MiB           1                   show_results_bound(prof)
  1190     44.2 MiB      0.0 MiB           1                   return val

In [ ]:
print('Memory usage of Floyd-Warshall with 100 edges:')
%load_ext memory_profiler
%mprun -f  floyd_warshall_space floyd_warshall_space(g)

Memory usage of Floyd-Warshall with 100 edges:
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
Filename: /Users/polina/Desktop/pythonProject/fancy.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    12     44.3 MiB     44.3 MiB           1   @profile
    13                                         def floyd_warshall_space(graph):
    14     44.3 MiB      0.0 MiB           1       num_vertices = len(graph)
    15     44.3 MiB      0.0 MiB       10303       dist = [[float('inf') if i != j else 0 for j in range(num_vertices)] for i in range(num_vertices)]
    16                                         
    17     44.3 MiB      0.0 MiB         101       for u in range(num_vertices):
    18     44.3 MiB      0.0 MiB       10100           for v in range(num_vertices):
    19     44.3 MiB      0.0 MiB       10000               if graph[u][v] != 0:
    20     44.3 MiB      0.0 MiB         100                   dist[u][v] = grap

Filename: /Users/polina/Desktop/pythonProject/venv/lib/python3.11/site-packages/memory_profiler.py

Line #    Mem usage    Increment  Occurrences   Line Contents
  1185     44.3 MiB     44.3 MiB           1               @wraps(wrapped=func)
  1186                                                     def wrapper(*args, **kwargs):
  1187     44.3 MiB      0.0 MiB           1                   prof = get_prof()
  1188     44.3 MiB     -0.0 MiB           1                   val = prof(func)(*args, **kwargs)
  1189     44.3 MiB      0.0 MiB           1                   show_results_bound(prof)
  1190     44.3 MiB      0.0 MiB           1                   return val

In [ ]:
# 1000 edges

g = graph(100,1000,1000)
print('Memory usage of Johnson with 1000 edges:')
%load_ext memory_profiler
%mprun -f  johnson_space johnson_space(g)

Memory usage of Johnson with 1000 edges:
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
Filename: /Users/polina/Desktop/pythonProject/fancy.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     6     44.8 MiB     44.8 MiB           1   @profile
     7                                         def johnson_space(g):
     8     44.8 MiB      0.0 MiB           1     graph = csr_matrix(g)
     9     44.8 MiB      0.0 MiB           1     dist_matrix, predecessors = johnson(csgraph=graph, directed=True, return_predecessors=True)
    10     44.8 MiB      0.0 MiB           1     return dist_matrix





Filename: /Users/polina/Desktop/pythonProject/venv/lib/python3.11/site-packages/memory_profiler.py

Line #    Mem usage    Increment  Occurrences   Line Contents
  1185     44.4 MiB     44.4 MiB           1               @wraps(wrapped=func)
  1186                                                     def wrapper(*args, **kwargs):
  1187     44.4 MiB      0.0 MiB           1                   prof = get_prof()
  1188     44.8 MiB      0.4 MiB           1                   val = prof(func)(*args, **kwargs)
  1189     44.8 MiB      0.0 MiB           1                   show_results_bound(prof)
  1190     44.8 MiB      0.0 MiB           1                   return val

In [ ]:
print('Memory usage of Floyd-Warshall with 1000 edges:')
%load_ext memory_profiler
%mprun -f  floyd_warshall_space floyd_warshall_space(g)

Memory usage of Floyd-Warshall with 1000 edges:
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
Filename: /Users/polina/Desktop/pythonProject/fancy.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    12     44.9 MiB     44.9 MiB           1   @profile
    13                                         def floyd_warshall_space(graph):
    14     44.9 MiB      0.0 MiB           1       num_vertices = len(graph)
    15     44.9 MiB      0.0 MiB       10303       dist = [[float('inf') if i != j else 0 for j in range(num_vertices)] for i in range(num_vertices)]
    16                                         
    17     44.9 MiB      0.0 MiB         101       for u in range(num_vertices):
    18     44.9 MiB      0.0 MiB       10100           for v in range(num_vertices):
    19     44.9 MiB      0.0 MiB       10000               if graph[u][v] != 0:
    20     44.9 MiB      0.0 MiB        1000                   dist[u][v] = gra

Filename: /Users/polina/Desktop/pythonProject/venv/lib/python3.11/site-packages/memory_profiler.py

Line #    Mem usage    Increment  Occurrences   Line Contents
  1185     44.9 MiB     44.9 MiB           1               @wraps(wrapped=func)
  1186                                                     def wrapper(*args, **kwargs):
  1187     44.9 MiB      0.0 MiB           1                   prof = get_prof()
  1188     27.9 MiB    -17.0 MiB           1                   val = prof(func)(*args, **kwargs)
  1189     28.4 MiB      0.5 MiB           1                   show_results_bound(prof)
  1190     28.4 MiB      0.0 MiB           1                   return val

In [ ]:
# 9800 edges

g = graph(100,9800,1000)
print('Memory usage of Johnson with 1000 edges:')
%load_ext memory_profiler
%mprun -f  johnson_space johnson_space(g)

Memory usage of Johnson with 1000 edges:
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
Filename: /Users/polina/Desktop/pythonProject/fancy.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     6     37.0 MiB     37.0 MiB           1   @profile
     7                                         def johnson_space(g):
     8     38.9 MiB      1.8 MiB           1     graph = csr_matrix(g)
     9     39.3 MiB      0.5 MiB           1     dist_matrix, predecessors = johnson(csgraph=graph, directed=True, return_predecessors=True)
    10     39.3 MiB      0.0 MiB           1     return dist_matrix





Filename: /Users/polina/Desktop/pythonProject/venv/lib/python3.11/site-packages/memory_profiler.py

Line #    Mem usage    Increment  Occurrences   Line Contents
  1185     36.7 MiB     36.7 MiB           1               @wraps(wrapped=func)
  1186                                                     def wrapper(*args, **kwargs):
  1187     36.7 MiB      0.0 MiB           1                   prof = get_prof()
  1188     39.3 MiB      2.6 MiB           1                   val = prof(func)(*args, **kwargs)
  1189     39.4 MiB      0.0 MiB           1                   show_results_bound(prof)
  1190     39.4 MiB      0.0 MiB           1                   return val

In [ ]:
print('Memory usage of Floyd-Warshall with 1000 edges:')
%load_ext memory_profiler
%mprun -f  floyd_warshall_space floyd_warshall_space(g)

Memory usage of Floyd-Warshall with 1000 edges:
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
Filename: /Users/polina/Desktop/pythonProject/fancy.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    12     39.6 MiB     39.6 MiB           1   @profile
    13                                         def floyd_warshall_space(graph):
    14     39.6 MiB      0.0 MiB           1       num_vertices = len(graph)
    15     39.8 MiB      0.2 MiB       10303       dist = [[float('inf') if i != j else 0 for j in range(num_vertices)] for i in range(num_vertices)]
    16                                         
    17     39.8 MiB      0.0 MiB         101       for u in range(num_vertices):
    18     39.8 MiB      0.0 MiB       10100           for v in range(num_vertices):
    19     39.8 MiB      0.0 MiB       10000               if graph[u][v] != 0:
    20     39.8 MiB      0.0 MiB        9787                   dist[u][v] = gra

Filename: /Users/polina/Desktop/pythonProject/venv/lib/python3.11/site-packages/memory_profiler.py

Line #    Mem usage    Increment  Occurrences   Line Contents
  1185     39.6 MiB     39.6 MiB           1               @wraps(wrapped=func)
  1186                                                     def wrapper(*args, **kwargs):
  1187     39.6 MiB      0.0 MiB           1                   prof = get_prof()
  1188     39.8 MiB      0.2 MiB           1                   val = prof(func)(*args, **kwargs)
  1189     39.8 MiB      0.0 MiB           1                   show_results_bound(prof)
  1190     39.8 MiB      0.0 MiB           1                   return val